## Generating initial text files for maizsim simulations

Maizsim has a total of X intput files. The files listed here are only ones that require: <br/>
1. customization for individual site-years, or
2. multiple sets for parametetr sensitivity testing

In [282]:
# Load in necessary packages
import os
import pandas as pd
import numpy as np
import glob as glob

In [246]:
# simple function that converts tuple into stirng
def tup_convert(tup):
    str = ''.join(tup)
    return str

### 0.0 Identifying parameters to test for:
- planting date (4/1 - 6/30, weekly interval: 12)
- juvenile leaf number (10-20, single leaf interval: 10)

In [276]:
# 191011: still need to think more about this
#         having emsemble of simulations will be quite tricky to manage everything
#         at the moment, maybe it's best to work on carrying out a single geo-spatial wide simulation across the sites I have
#         and add in the emsemble element once I get that going

### 0.1 Making directories to hold initial files & simulation outputs
Repository structure: <br\>
- init_files/
    - specific siteyears/ 
        - parameter combinations/ (191011: holding off this part at the moment)

In [345]:
# This cell only needs to be executed once because after that the folders will be created
# if you try executing this code block after the folders exist it won't execute and will throw an error
siteyears = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/site_year_crithr1.csv', dtype=str, index_col=0)
for i in np.arange(0,len(siteyears)):
    os.mkdir('/home/disk/eos8/ach315/upscale/init_files/site_years/' + str(siteyears.iloc[i,0]) + '_' + str(siteyears.iloc[i,1]))
    
for i in np.arange(0,len(siteyears)):
    os.mkdir('/home/disk/eos8/ach315/upscale/runs/' + str(siteyears.iloc[i,0]) + '_' + str(siteyears.iloc[i,1]))

### 0.2 Identifying initial files that need to customized for each site-year due to time & date
1. init.txt
2. time.txt
3. climate.txt
4. management.txt

### 0.3 Specifying some general input variables

In [419]:
directories = glob.glob('/home/disk/eos8/ach315/upscale/init_files/site_years/*')
site_info = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/site_info.csv', dtype=str, index_col=0)
year = i.split('/')[-1].split('_')[-1]
site = i.split('/')[-1].split('_')[-2]
lat = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lat
lon = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lon
tab = '    '

### 0.4 Create a function that generates initial files for maizsim

In [ ]:
# 191018 JH: still working on this
def write_txt(file_name, file_extension, file_mode, strings, directory):
    for i in directory: 
        # opening empty files in preparation for writing
        file_name = open(i + '/' + file_name + '.', file_extension, str(file_mode))
        # putting strings together
        file_name.writelines(strings)
        # closing file
        file_name.close()
    

In [421]:
strings = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11, str12]

In [422]:
strings

['***Standard Meteorological Data\n',
 'latitude\n',
 '42.33\n',
 'daily bulb temp, daily wind, rain intensity, daily conc, furrow, relitive humidity, co2\n',
 '0    0    0    0    0    1    1\n',
 'parameters for changing of units: BSOLAR BTEMP ATEMP ERAIN BWIND BIR\n',
 'BSOLAR is 1e6/3600 to go from jm-2h-1 to wm-2\n',
 '3600    1    0    0    1    1\n',
 'average values for the site\n',
 'WINDA    IRAV    Conc    CO2\n',
 '3    6    0    400\n',
 '0    1']

### 1. init.txt
iniital file for general setup

In [298]:
### --- constant parameters ---

# management
poprow = 7.5 # plants/row
rowsp = 75 
plant_density = 10 # plant density (___)
rowang = 0
x_seed = 0
y_seed = 195
cec = 0.65
eomult = 0.5
co2 = -1

# output soil file or not
soil_yes = ('0', tab, '1')
soil_no = ('1', tab, '0')

# timing
timestep = 60

In [ ]:
### --- text strings ---

str1 = '***Initialization data\n'
str2 = tup_convert(['poprow', tab, 'rowsp', tab, 'rowsp', tab, 'plant_density', tab, 
                    'rowang', tab, 'x_seed', tab, 'y_see', tab, 'eomult', tab, 'co2\n'])
str3 = tup_convert(['%.1f' %poprow, tab, '%.1f' %rowsp, tab, '%.1f' %plant_density, tab,
                    '%.1f' %rowang, tab, '%.1f' %x_seed, tab, '%.1f' %y_seed, tab, '%.2f' %cec, tab,
                    '%.2f' %eomult, tab, '%.0f\n' %co2])
str4 = tup_convert(['latitude', tab, 'longitude', tab, 'altitude\n'])
str5 = tup_convert(['%.2f' %lat, tab, '%.2f' %lon, tab, '%.2f\n' %alt])
str6 = 'autoirrigate\n'
str7 = '%.0f\n' %irrigate
str8 = tup_convert(['begin', tab, 'sowing', tab, 'end', tab, 'timestep (mins)\n'])
str9 = tup_convert([start, tab, sowing, tab, end, tab, '%.0f\n' %timestep])
str10 = 'output soils data (g03, g04, g05, and g06 files) 1 if true\n'
str11 = tup_convert(['no soil files', tab, 'outputsoil files\n'])
str12 = tup_convert(soil_yes)

strings = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11, str12]

In [376]:
### --- creating an init.txt file for each site-year ---

for i in directories:
    init = open(i + '/init.txt', 'w')
    year = i.split('/')[-1].split('_')[-1]
    site = i.split('/')[-1].split('_')[-2]
    
    # customized parameters: location
    lat = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lat
    lon = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lon
    alt = 0 # we actually don't have altitude information, but I'm also not sure how it affects the model
    # customized parameters: timing
    start = "'04/01/" + year + "'"
    sowing = "'04/01/" + year + "'"
    end = "'10/31/" + year + "'"
    
    # writing out text strings for each line in init.txt
    str1 = '***Initialization data\n'
    str2 = tup_convert(['poprow', tab, 'rowsp', tab, 'rowsp', tab, 'plant_density', tab, 
                       'rowang', tab, 'x_seed', tab, 'y_see', tab, 'eomult', tab, 'co2\n'])
    str3 = tup_convert(['%.1f' %poprow, tab, '%.1f' %rowsp, tab, '%.1f' %plant_density, tab,
                        '%.1f' %rowang, tab, '%.1f' %x_seed, tab, '%.1f' %y_seed, tab, '%.2f' %cec, tab,
                        '%.2f' %eomult, tab, '%.0f\n' %co2])
    str4 = tup_convert(['latitude', tab, 'longitude', tab, 'altitude\n'])
    str5 = tup_convert(['%.2f' %lat, tab, '%.2f' %lon, tab, '%.2f\n' %alt])
    str6 = 'autoirrigate\n'
    str7 = '%.0f\n' %irrigate
    str8 = tup_convert(['begin', tab, 'sowing', tab, 'end', tab, 'timestep (mins)\n'])
    str9 = tup_convert([start, tab, sowing, tab, end, tab, '%.0f\n' %timestep])
    str10 = 'output soils data (g03, g04, g05, and g06 files) 1 if true\n'
    str11 = tup_convert(['no soil files', tab, 'outputsoil files\n'])
    str12 = tup_convert(soil_yes)

    # putting all strings together
    L = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11, str12]
    
    # writing out lines in init.txt and clsoing file
    init.writelines(L)
    init.close()

### 2. time.txt

In [405]:
#--- constant parameters
dt = 0.0001
dt_min = 1e-7
dmul1 = 1.3
dmul2 = 0.3
daily_output = '1    0\n'
hourly_output = '0    1\n'
daily_intput = '1    0\n'
hourly_input = '0    1\n'

#--- adjustable parameters: time
start = '04/01/'
end = '10/31/'

#--- adjustable parameters: output format
output_timestep = hourly_output
weadata_timestep = hourly_input

In [406]:
# finding where all the time.txt files should go for individual site-years
directories = glob.glob('/home/disk/eos8/ach315/upscale/init_files/site_years/*')

# creating an init.txt file for each site-year
for i in directories:
    time = open(i + '/time.txt', 'w')
    year = i.split('/')[-1].split('_')[-1]
    
    # customized parameters: timing
    init_time = "'" + start + year + "'"
    end_time = "'" + end + year + "'\n"
    
    # text strings
    str1 = '***Synchronizer Information\n'
    str2 = tup_convert(['initial time', tab, 'dt', tab, 'dtmin', tab, 'dmul1', tab, 'dmul2', tab, 'tfin\n'])
    str3 = tup_convert([init_time, tab, '%.4f' %dt, tab, '%.7f' %dt_min, tab, '%.2f' %dmul1, tab, '%.2f' %dmul2, tab, end_time])
    str4 = 'output variables, 1 if true    Daily    Hourly\n'
    str5 = output_timestep
    str6 = 'weather data, 1 if true    Daily     Hourly\n'
    str7 = weadata_timestep
    
    # putting text strings together
    L = [str1, str2, str3, str4, str5, str6, str7]
    
    # writing out lines in init.txt and clsoing file
    time.writelines(L)
    time.close()

### 3. climate.txt
inital file for basic climate info

In [417]:
#--- constant parameters
bsolar = 3600
btemp = 1
atemp = 0
erain = 0.1
bwind = 1
bir = 1

# climate data input
climate_input = tup_convert(['0', tab, '0', tab, '0', tab, '0', tab, '0', tab, '1', tab, '1\n'])

# average values for site
# 19118: I'm actually not sure how these values are calculated and how important they are?
winda = 3
irav = 6
conc = 0
co2 = 400

In [418]:
# finding where all the climate.txt files should go for individual site-years




# creating an init.txt file for each site-year
for i in directories:
    climate = open(i + '/climate.txt', 'w')
    site = i.split('/')[-1].split('_')[-2]
    site_info = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/site_info.csv', dtype=str, index_col=0)
    lat = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lat

    # text strings
    str1 = '***Standard Meteorological Data\n'
    str2 = 'latitude\n'
    str3 = '%.2f\n' %lat
    str4 = 'daily bulb temp, daily wind, rain intensity, daily conc, furrow, relitive humidity, co2\n'
    str5 = climate_input
    str6 = 'parameters for changing of units: BSOLAR BTEMP ATEMP ERAIN BWIND BIR\n'
    str7 = 'BSOLAR is 1e6/3600 to go from jm-2h-1 to wm-2\n'
    str8 = tup_convert(['%.0f' %bsolar, tab, '%.0f' %btemp, tab, '%.0f' %atemp, tab, '%.0f' %erain, tab, '%.0f' %bwind, tab, '%.0f\n' %bir])
    str9 = 'average values for the site\n'
    str10 = tup_convert(['WINDA', tab, 'IRAV', tab, 'Conc', tab, 'CO2\n'])
    str11 = tup_convert(['%.0f' %winda, tab, '%.0f' %irav, tab, '%.0f' %conc, tab, '%.0f\n' %co2])
    
    # putting strings together
    L = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11]
    
    # writing out lines in init.txt and clsoing file
    climate.writelines(L)
    climate.close()

### 4. management.txt

### 5. cultivar.txt
initial file for cultivar and soil traits

In [ ]:
cultivar = open('/home/disk/eos8/ach315/upscale/init_files/cultivar.txt', 'w')

In [231]:
# parameters: cultivar traits
gdd_mature = 1600
juv_leaves = 15
deylength_sensitivity = 0
rmax_ltar = 0.53 # maximum leaf tip appearance rate (leaves/day)
rmax_ltir = 0.978 # maximum leaf tip iniitation rate (leaves/day)
phyllochron = 3

# paramters: soil water
(rrm, rrry, rvrl) = (166.7, 31.3, 0.73)
(alpm, alpy, rtwl, rtminwtperunitarea) = (0.35, 0.04, 1.059e-004, 2.0000e-004)
(wl, wa, wr, wb) = (1, 1, 2, 10)

# parameter: soil nitrogen
(isink, rroot) = (0, 0.017)
(consti, constk, cmin0) = (35.0, 0.5, 0.01)

In [ ]:
# putting text strings together
str1 = 'corn growth simulation for '
str2 = 'cultivar: _____'
str3 = 'GDD'
str4 = 'maturity'
str5 = 


In [ ]:
cultivar.writelines(L)
cultivar.close()

### 4. management.txt
iniital file for management

In [ ]:
management = open('/home/disk/eos8/ach315/upscale/init_files/management.txt', 'w')

In [ ]:
management.writelines(L)
management.close()

### 5. time.txt

In [237]:
names = ['test', 'test1', 'test2']

In [239]:
mkdir test